In [ ]:
import sys
import gensim
import pandas as pd
import sklearn
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords  
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
TaggededDocument = gensim.models.doc2vec.TaggedDocument
path = "/content/drive/My Drive"
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

 
def get_datasest():
    file=open("Trec_microblog11.txt",encoding='utf-8')
    line=file.readline()
    docs=[]
    while line:
        words=word_tokenize(line)
        sentence=[]
        for i in range(1,len(words)):
            sentence.append(words[i])
        docs.append(sentence)
        line = file.readline()
    x_train=[]
    for i, text in enumerate(docs):
        document = TaggededDocument(text, tags=[i])
        x_train.append(document)
  
    return x_train
def getVecs(model, corpus, size):
    vecs = [np.array(model.docvecs[z.tags[0]].reshape(1, size)) for z in corpus]
    return np.concatenate(vecs)
 
def train(x_train, size=5000, epoch_num=1):
    model_dm = Doc2Vec(x_train,min_count=1, window = 5, size = size, sample=1e-5, workers=4)
    import logging
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 
    model_dm.train(x_train, total_examples=model_dm.corpus_count, epochs=70)
    model_dm.save('csi4107A2')
 
    return model_dm


x_train = get_datasest()
model_dm = train(x_train)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2021-03-21 13:31:29,679 : WARNING : Effective 'alpha' higher than previous training cycles
2021-03-21 13:31:29,680 : INFO : training model with 4 workers on 118899 vocabulary and 5000 features, using sg=0 hs=0 sample=1e-05 negative=5 window=5
2021-03-21 13:31:30,825 : INFO : EPOCH 1 - PROGRESS: at 5.52% examples, 20136 words/s, in_qsize 8, out_qsize 0
2021-03-21 13:31:31,933 : INFO : EPOCH 1 - PROGRESS: at 14.27% examples, 26881 words/s, in_qsize 7, out_qsize 0
2021-03-21 13:31:33,037 : INFO : EPOCH 1 - PROGRESS: at 23.02% examples, 28626 words/s, in_qsize 7, out_qsize 0
2021-03-21 13:31:34,231 : INFO : EPOCH 1 - PROGRESS: at 31.72% examples, 28926 words/s, in_qsize 8, out_qsize 0
2021-03-21 13:31:35,392 : INFO 

In [ ]:
import copy
import sys
import gensim
import pandas as pd
import sklearn
import numpy as np
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
import re
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
TaggededDocument = gensim.models.doc2vec.TaggedDocument
path = "/content/drive/My Drive"
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)
def test(test_text,length):
    model_dm = Doc2Vec.load("csi4107A2")
    inferred_vector_dm = model_dm.infer_vector(test_text)
    sims = model_dm.docvecs.most_similar([inferred_vector_dm], topn=length)
    
    return sims
def dealwithquery(filepath):
    f = open(filepath)
    line = f.readline()
    query={}
    j=0
    while line:
       thisquery=[]
    
       if ('<title>' in line):
         j=j+1 
         line=line.strip('<title>')
         line=line.strip('</title>\n')
         words=word_tokenize(line)    
         for i in words:
            thisquery.append(i)
         query[j]=thisquery
       else:
         line = f.readline()
    return query
def getindex():
    Indexing = {}
    file=open("Trec_microblog11.txt",encoding='utf-8')
    line=file.readline()
    num=0
    while line:
        words=word_tokenize(line)
        Indexing[num]=words[0]
        line = file.readline()
        num=num+1
        

    length=len(Indexing)
    
    return (Indexing,length)

def reclean(path):
  Indexing = {}
  f2 = open(path,"r")
  lines = f2.readlines()
  for line3 in lines:
    a=line3.split(' ')[0]
    if (a in Indexing):
      b=Indexing[a]
      b.append(line3.split(' ')[2])
      Indexing[a]=b  
    else:
      Indexing[a]=[line3.split(' ')[0]]     
  return Indexing
def main():
    qnum=0
    query=dealwithquery('topics_MB1-49.txt')
    (id,length)=getindex()
    with open('doc2vec_new_output.txt','w') as fa:
      for o in range(len(query)):
        qnum=qnum+1
        sims=test(query[o+1],length)
        rank=0
        a=reclean('output.txt')
        b=list(a.values())[qnum-1]
        c=copy.deepcopy(b)
        for (i,j) in sims:
            if(id[i] in b):
              b.remove(id[i])
              oldindex=c.index(id[i])
              rank=rank+1
              output=str(qnum)+" Q0 "+str(id[i])+" "+str(rank)+" "+str(j)+" query "+str(qnum)+" N0."+str(rank) +" old Ranking is "+str(oldindex+1)+"\n"
              fa.write(output)
              print(output)
    fa.close()
    
main()

流式输出内容被截断，只能显示最后 5000 行内容。
45 Q0 30026649533677568 129 0.8694837689399719 query 45 N0.129 old Ranking is 103

45 Q0 32164401125531648 130 0.8692599534988403 query 45 N0.130 old Ranking is 184

45 Q0 32498547156852736 131 0.8692387342453003 query 45 N0.131 old Ranking is 419

45 Q0 30180072052756481 132 0.8690590858459473 query 45 N0.132 old Ranking is 773

45 Q0 30134727516098561 133 0.8687537908554077 query 45 N0.133 old Ranking is 412

45 Q0 32349041220124672 134 0.8683815002441406 query 45 N0.134 old Ranking is 153

45 Q0 34635104106053633 135 0.8682808876037598 query 45 N0.135 old Ranking is 91

45 Q0 32110774721708032 136 0.8677235245704651 query 45 N0.136 old Ranking is 517

45 Q0 30601497439899648 137 0.8674269914627075 query 45 N0.137 old Ranking is 337

45 Q0 31015910500204544 138 0.8670072555541992 query 45 N0.138 old Ranking is 241

45 Q0 32210872642310144 139 0.8662599325180054 query 45 N0.139 old Ranking is 125

45 Q0 30316358721867776 140 0.8658924698829651 query 45 N0.14